# [How to pass tool outputs to chat models](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) ->int:
    """add a and b."""
    return a + b

@tool
def multiply(a: int, b: int) ->int:
    """multiply a and b."""
    return a * b

tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

Now, let's get the model to call a tool. We'll add it to a list of messages that we'll treat as conversation history:

In [4]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

# print(ai_msg)
print(ai_msg.tool_calls)

messages.append(ai_msg)
print(messages)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BvTzh4qreDsIcEhwuqgIrjzR', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_svzsgBIcLiVgTb70XKgUhnlG', 'type': 'tool_call'}]
[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BvTzh4qreDsIcEhwuqgIrjzR', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_svzsgBIcLiVgTb70XKgUhnlG', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 85, 'total_tokens': 135, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-885912

Next let's invoke the tool functions using the args the model populated!

Conveniently, if we invoke a LangChain `Tool` with a `ToolCall`, we'll automatically get back a `ToolMessage` that can be fed back to the model:



In [5]:
tool_list = {"add": add, "multiply": multiply}

for tool_call in ai_msg.tool_calls:
    # selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    selected_tool = tool_list[tool_call["name"].lower()]

    print(selected_tool)

    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

name='multiply' description='multiply a and b.' args_schema=<class 'langchain_core.utils.pydantic.multiply'> func=<function multiply at 0x10716cb80>
name='add' description='add a and b.' args_schema=<class 'langchain_core.utils.pydantic.add'> func=<function add at 0x10716caf0>


[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BvTzh4qreDsIcEhwuqgIrjzR', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_svzsgBIcLiVgTb70XKgUhnlG', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 85, 'total_tokens': 135, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-88591266-e666-4d68-b51f-7fa08bbadb19-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_BvTzh4qreDsIcEhwuqgIrjzR', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_sv

In [16]:
res = llm_with_tools.invoke(messages)
print(res)

content='The result of \\(3 \\times 12\\) is \\(36\\), and the result of \\(11 + 49\\) is \\(60\\).' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 151, 'total_tokens': 184, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f59a81427f', 'finish_reason': 'stop', 'logprobs': None} id='run-60d39f42-0082-4fac-9554-00e82ef3165f-0' usage_metadata={'input_tokens': 151, 'output_tokens': 33, 'total_tokens': 184}
